# Oracle 전용
> 국민대학교 김남규 교수 강의:
>
> https://www.youtube.com/playlist?list=PLg_wJlcMiuKtGdlIaAZ0rOPPQuTDENnEQ
## 1. 실습 환경 (택 1)
- Local 환경
    - Oracle XE와 SQL Developer 설치
    - 속도가 빠르며 Query 길이에 제한이 없음
    - 설치가 복잡함
- Cloud 환경
    - https://livesql.oracle.com
    - 설치가 필요 없으며 어디에서나 접속 가능
    - 속도가 느리며 Query 길이에 제한이 있음

DUAL
- 시스템이 제공해 주는 보이지 않는 테이블
- DUMMY라는 컬럼명 한 개에 'X'라는 데이터 하나가 들어있다.
- 예) SELECT 'ABC', 5+4
      FROM DUAL;
    - 출력: 
        ```
           +---+------+-----+
           |   |'ABC' | 5+4 |
           +---+------+-----+
           | 1 | ABC  | 9   |
           +---+------+-----+
        ```

NULL 값
- ORACLE에서 NULL은 가장 큰 값으로 취급 됨 
    - MySQL, Microsoft의 sql server는 반대로 NULL을 가장 작은 값으로 취급
    - -> ORDER BY 컬럼명 ASC 했을 때 NULL값들은 가장 아래에 출력됨

사용자 생성 객체 확인
-   ```mysql
    SELECT * 
    FROM USER_OBJECTS    # USER_OBJECTS는 시스템이 저절로 만들어준 테이블
    WHERE OBJECT_TYPE = 'TABLE';
    ```
    -> 생성된 모든 테이블 출력

타입 (data type)
- 테이블의 타입 확인:
    - DESCRIBE 테이블명
    - 또는 DESC 테이블명
- 문자 CHAR(숫자), VARCHAR2(숫자)
    - CHAR(숫자)
        - 고정 문자열 (기본 1 Byte ~ 최대 2000 Byte)
        - 변수에 할당된 값이 고정 길이보다 작은 경우, 나머지 공간에 공백이 채워짐
        - 'AA' = 'AA '
    - VARCHAR2(숫자)
        - 가변 길이 문자열 (기본 1 Byte ~ 최대 4000 Byte)
        - 변수에 할당된 값이 고정 길이보다 작은 경우, 나머지 공간은 사용하지 않음
        - 'AA' <> 'AA '
- 숫자 NUMBER[(숫자[, 숫자])]
    - 정수, 실수 등의 숫자 정보
    - MS SQL Server의 경우 10가지 이상의 숫자 타입 지원
    - 전체 자리수와 소수 부분 자리수를 명시함
    - 예) NUMBER(8, 2) -> 정수 부분 6자리, 소수 부분 2자리
- 날짜 DATE
    - 날짜와 시간 정보
제약조건
- 종류:
    - PRIMARY KEY (기본키): 기본키 정의 (테이블 당 하나의 기본키 제약만 정의 가능); (NOT NULL, UNIQUE가 자동으로 같이 설정 됨)
        - 컬럼이 2개 이상 묶여 있는 하나의 PRIMARY KEY 설정 가능
        - 예) CREATE TABLE schedule (
            stadium_id             CHAR(3) NOT NULL,
            sche_date              CHAR(8) NOT NULL,
            home_score             NUMBER(2),
            away_score             NUMBER(2),
            CONSTRAINT             schedule_pk PRIMARY KEY (stadium_id, sche_date),
            CONSTRAINT             schedule_fk PRIMARY KEY (stadium_id), REFERENCES stadium (stadium_id)
            );
    - FOREIGN KEY (외래키): 다른 테이블의 기본키를 참조하는 외래키를 지정; 외래키 지정시 참조 무결성 제약 옵션 선택
    - NOT NULL: NULL 값 입력 불허용
    - UNIQUE: 해당 컬럼의 값이 테이블 내에서 유일해야 함을 제약
    - CHECK: 입력할 수 있는 값을 제한
        - 예) CONSTRAINT back_no_ck CHECK (back_no < 99 AND back_no > 10)
- 제약 조건 지정 3가지 방식 (테이블 생성시)
    - 제약 조건명을 명시적으로 부여 가능, 묵시적으로 제약 조건명 없이 제약 조건 설정 가능 (이럴 경우는 시스템에서 알아서 제약조건명을 부여해 주는 것)
    1. 묵시적 방식 (CONSTRAINT의 이름을 지정해주지 않고 컴퓨터가 알아서 지정해줌) - 가장 간단
        - 예) CREATE TABLE player1 (
                        player_id CHAR(7) PRIMARY KEY,
                        player_name VARCHAR(20) NOT NULL,
                        nickname VARCHAR(30) UNIQUE,
                        height NUMBER(3) CHECK (height >= 150 AND height <= 200),
                        team_id CHAR(3) REFERENCES team(team_id)
                );
    2. 명시적 방식 1 (CONSTRAINT의 이름을 지정해 줌)
        - 예) CREATE TABLE player2 (
                        player_id CHAR(7) CONSTRAINT p2_pk_id PRIMARY KEY,
                        player_name VARCHAR2(20) CONSTRAINT p2_nn_name NOT NULL,
                        nickname VARCHAR2(30) CONSTRAINT p2_un_nick UNIQUE,
                        height NUMBER(3) CONSTRAINT p2_ck_height CHECK (height >= 150 AND height <= 200),
                        team_id CHAR(3) CONSTRAINT p2_fk_tid REFERENCES team(team_id)
                );
    3. 명시적 방식 2
        - 예) CREATE TABLE player3 (
                        player_id CHAR(7),
                        player_name VARCHAR2(20) CONSTRAINT p3_nn_name NOT NULL,
                        nickname VARCHAR2(30),
                        height NUMBER(3),
                        team_id CHAR(3),

                        CONSTRAINT p3_pk_id PRIMARY KEY (player_id),
                        CONSTRAINT p3_un_nk UNIQUE (nickname),
                        CONSTRAINT p3_ck_height CHECK (height >= 150 AND height <= 200),
                        CONSTRAINT p3_fk_tid FOREIGN KEY (team_id) REFERENCES team(team_id)
                );
- 제약 조건 확인
    - SELECT *
        FROM ALL_CONSTRAINTS          # ALL_CONSTRAINTS는 시스템이 저절로 만들어준 테이블
        WHERE table_name IN ('player1', 'player2', 'player3')
        ORDER BY CONSTRAINT_NAME;
- FK 제약 조건 옵션 설정
    - 예) CONSTRAINT fk1 FOREIGN KEY (team_id) REFERENCES team(team_id)
            ON DELETE CASCADE ON UPDATE RESTRICT;
    - Referential Triggered Action
        - ON UPDATE -> 참조되는 키가 UPDATE 될 때 할 Action 설정
        - ON DELETE -> 참조되는 키가 DELETE 될 때 할 Action 설정
    - Referential Action (뭘 할 것인지)
        - RESTRICT (Default): 기본값의 삭제/갱신을 불허 -> 가장 대표적인 방법
        - NO ACTION: RESTRICT와 동일하게 동작
        - CASCADE:
            - 기본키가 삭제되면 해당 값을 외래키로 갖는 레코드(모든 행)도 삭제
            - 기본키가 갱신되면 이를 참조하는 외래키를 새로운 값으로 갱신
        - SET NULL:
            - 기본키가 삭제/갱신되면 이를 참조하는 외래키를 NULL로 갱신
- 기존 테이블을 활용한 테이블 생성 (SELECT문 활용)
    - CREATE TABLE player_temp AS SELECT * FROM player;
        - player의 모든 레코드로 player_temp라는 테이블을 만드는 것
    - 참조하여 테이블을 만들 때 제약조건은 NOT NULL만 복제됨
        - PK, FK, UNIQUE, CHECK 등은 수동(직접)으로 추가해야 됨

ORACLE 연산자
- 연산자 우선순위
    1. 괄호 ()
    2. NOT 연산자
    3. 비교 연산자, SQL 연산자
    4. AND
    5. OR
- 산술 연산자
    - +, -, *, /, %(나머지)
    - number와 date 자료형에 적용 (MySQL에서는 날짜 시간에는 바로 적용 안 됨 )
    - date에 적용 시 날짜 값으로 적용됨 (23/01/29인 값에 +3 하면 23/02/01이 출력됨)
    - Null 값에 산술 연산의 결과는 항상 NULL
- 비교 연산자
    - >, <, >=, <=, =
    - 모든 자료형에 적용 (문자열 크기는 사전순 - 먼저 나오는 것이 작음)
        - 예) '01' < '02' < '1' < '11' < '2'  
    - !=(같지 않음), <>(같지 않음), 
    - !< (왼쪽 값이 오른쪽 값보다 작지 않음)
    - !> (왼쪽 값이 오른쪽 값보다 크지 않음)
    - NULL에는 비교 연산자 사용 불가
- 논리 연산자
    - 모든 자료형에 적용
    - NOT, AND, OR
    - 진리표
        - AND (*와 같은 개념)
            - 참 AND 참 = 참
            - 참 AND 거짓 = 거짓
            - 거짓 AND 참 = 거짓
            - 거짓 AND 거짓 = 거짓
            - 참 AND NULL = NULL
            - 거짓 AND NULL = 거짓
            - NULL AND NULL = NULL
        - OR (+와 같은 개념)
            - 참 OR 참 = 참
            - 참 OR 거짓 = 참
            - 거짓 OR 참 = 참
            - 거짓 OR 거짓 = 거짓
            - 참 OR NULL = 참
            - 거짓 OR NULL = NULL
            - NULL OR NULL = NULL
        - NOT
            - 참 = NOT 거짓
            - 거짓 = NOT 참
            - NULL = NOT NULL
- SQL 연산자
    - 합성(연결) 연산자 - 문자열 결합  
        - || (파이프): str1 || str2
            - 2개 이상 가능: str1 || str2 || str3 || str4 ...
                - 작동하는 방법: (((str1 || str2) || str3) || str4) ... (이항?)
            - 예) str = 'a' || 'bcd'
            >> str = 'abcd'
        - CONCAT(str1, str2)
            - 2개만 연결 - 3개 이상 불가능
    - BETWEEN + AND, NOT BETWEEN + AND
        - 예) SELECT name FROM author WHERE id BETWEEN 1 AND 3
    - IN (list), NOT IN (list)
        - 예) SELECT name FROM author WHERE id IN (1, 2, 3)
        - 예) SELECT name AS "작가 이름"
                FROM author
                WHERE (id, name) IN ((1, '김%'), (2, '이%'));
    - IS NULL, IS NOT NULL
        - 주의: id = NUlL 또는 id <> NULL 할 시 에러가 난다 (이유: NULL은 비교 연산자 사용 불가)
    - LIKE
        - 문자열 비교 연산
        - 와일드카드 사용 가능 
            - '%' 임의의 문자 N개 / '_' 임의의 문자 1개 ('__'는 임의의 문자 2개, '___'는 임의의 문자 3개, and so on)

DDL (Data Definition Lang) 데이터 정의어: 테이블/컬럼 정의/변경하는 명령어
- CREATE TABLE 테이블명 (필드이름1 필드타입1, 필드이름2 필드타입2, ...)
    - 테이블 생성
        - 다른 테이블의 값을 참조할 수 있음 (REFERENCES)
        - 예) CREATE TABLE stadium (
                        stadium_id           CHAR(3) NOT NULL,
                        stadium_name         VARCHAR2(40) NOT NULL,
                        hometeam_id          CHAR(3),
                        seat_count           NUMBER,
                        address              VARCHAR2(60),
                        tel                  VARCHAR2(10),
                        CONSTRAINT           stadium_pk PRIMARY KEY (stadium_id)
                );
        - 예) CREATE TABLE team (
                        team_id              CHAR(3) NOT NULL,
                        region_name          VARCHAR2(8) NOT NULL,
                        team_name            VARCHAR2(40) NOT NULL,
                        stadium_id           CHAR(3) NOT NULL,
                        address              VARCHAR2(60),
                        tel                  VARCHAR2(10),
                        CONSTRAINT           team_pk PRIMARY KEY (team_id),
                        CONSTRAINT           team_fk FOREIGN KEY (stadium_id) REFERENCES stadium (stadium_id)
                );
            - FOREIGN KEY 설정 시 해당 키의 값을 포함하고 있는 테이블이 먼저 생성되어 있어야 함 -> stadium_id를 갖고 있는 stadium이라는 테이블이 먼저 생성되어 있어야 함
    - 생성 규칙:
        - 테이블명: 단수형 권장, 다른 테이블과 중복 불가
        - 컬럼명: 테이블 내 중복 불가, 생성시 각 컬럼들은 괄호 내에서 콤마로 구분, 칼럼 뒤에 데이터 유형 지정 필수
        - 제약조건명(constraints): 다른 제약조건의 이름과 중복 불가 (제약: PRIMARY KEY, UNIQUE KEY, NOT NULL, CHECK, FOREIGN KEY 등)
        - 예약어(reserved word; 예, SELECT, FROM, WHERE, UPDATE, SET 등) 사용 불가
        - 문자, 숫자, 일부 기호 (_, $, #)만 허용
        - 반드시 문자로 시작 가능 (숫자, 기호 불가)
    - 기존 테이블을 활용한 테이블 생성 (SELECT문 활용)
        - CREATE TABLE player_temp AS SELECT * FROM player;
            - player의 모든 레코드로 player_temp라는 테이블을 만드는 것
        - 참조하여 테이블을 만들 때 제약조건은 NOT NULL만 복제됨
            - PK, FK, UNIQUE, CHECK 등은 수동(직접)으로 추가해야 됨
- RENAME문
    - 테이블명 변경
        - RENAME 원래테이블명 TO 새로운테이블명;
        - 예) RENAME author TO author_new;
- ALTER TABLE 테이블명
    - 수정 (TABLE 구성변경)
    - 컬럼 추가
        - ALTER TABLE 테이블명 ADD (새로운필드명 필드타입);
        - 예) ALTER TABLE player_temp ADD (address VARCHAR2(80));
        - ** MySQL에서는 ADD 뒤에 COLUMN도 들어감
            - MySQL: ALTER TABLE 테이블명 ADD COLUMN 새로운필드명 필드타입;
                - 예) ALTER TABLE author ADD COLUMN updated_at DATETIME;
    - 컬럼 정의 수정 
        - ALTER TABLE 테이블명 MODIFY ~
        - 허용되지 않는 경우: 이미 입력되어 있는 값에 영향을 미치는 변경은 불가능
        - 허용되는 경우:
            - 데이터 타입 변경 -> 테이블에 아무 행도 없거나, 해당 컬럼이 NULL만 갖고 있을 때 가능
            - 컬럼의 크기 변경
                - 컬럼의 크기 확대 -> 항상 가능
                - 컬럼의 크기 축소 -> 테이블에 아무 행도 없거나, 
                                    해당 컬럼이 NULL만 갖고 있거나,
                                    현재 저장된 값을 수용할 수 있는 크기로의 축소만 가능
            - DEFAULT 값 추가/수정 -> 추가/수정 이후 삽입되는 행에만 영향을 미침
            - 제약조건
                - NOT NULL
                    - NOT NULL 제약조건 추가 -> 테이블에 아무 행도 없거나, 해당 컬럼에 NULL이 없을 때 가능
                        - ALTER TABLE 테이블명 MODIFY (컬럼명 [NOT] NULL)
                    - NOT NULL 제약조건 삭제 -> 항상 가능
                - 다른 제약 조건들 -> 테이블 생성 후에도 제약조건 추가/삭제 가능
                    - 제약조건 확인
                        - SELECT * FROM ALL_CONSTRAINTS
                            WHERE table_name = 'player_temp';
                    - 제약조건 추가
                        - ALTER TABLE 테이블명 ADD CONSTRAINT 제약조건명 ~
                        - 예) player_temp 테이블에 team에 대한 FOREIGN KEY 추가
                            - ALTER TABLE player_temp
                                ADD CONSTRAINT player_temp_fk FOREIGN KEY (team_id) REFERENCES team (team_id);
                    - 제약조건 삭제
                        - ALTER TABLE 테이블명 DROP CONSTRAINT 제약조건명
                        - 예) player_temp 테이블에 team에 대한 FOREIGN KEY 
                            - ALTER TABLE player_temp
                                DROP CONSTRAINT player_temp_fk;
    - 컬럼 삭제
        - ALTER TABLE 테이블명 DROP COLUMN 삭제할컬럼명;
    - 제약 추가
        - ALTER TABLE 테이블명 ADD CONSTRAINT 제약명 제약조건(컬럼명);
        - 예) ALTER TABLE test_tab ADD CONSTRAINT const1 UNIQUE(col1);
        - 테이블(test_tab)의 컬럼(col1)에 const1이라는 제약명으로 UNIQUE 제약 설정
    - 제약 삭제
        - ALTER TABLE 테이블명 DROP CONSTRAINT 제약명;
        - 예) ALTER TABLE test_tab DROP CONSTRAINT const1;
        - 테이블(test_tab)에 설정되어있는 const1이라는 제약명을 가진 제약을 삭제한다
    - 제약 활성화/비활성화
        - ALTER TABLE 테이블명 [ENABLE/DISABLE] CONSTRAINT 제약명;
        - 예) ALTER TABLE test_tab DISABLE CONSTRAINT const1;
        - 테이블(test_tab)에 설정된 const1라는 제약명을 가진 제약을 일시적으로 비활성화한다
- DROP TABLE 테이블명
    - 삭제
        - 예) DROP TABLE team;
        - 그러나 다른 테이블이 team 테이블의 값을 참조하고 있을 경우 에러 발생할 것임
            - 예) player 테이블의 player_id 컬럼이 team 테이블의 player_id 컬럼을 foreign key로 참조하고 있을 경우
    - 관련된 제약조건을 함께 삭제
        - 예) DROP TABLE 테이블명 CASCADE CONSTRAINT;
DML (Data Manipulation Lang) 데이터 조작어: 데이터 조회/추가/수정/삭제
- SELECT 
    - 조회 - 조건이 없으면 모든 data 조회
    - 문법: SELECT [ALL/DISTINCT] 컬럼/필드명 FROM 테이블명 WHERE 조건
    - 컬럼/필드명을 명시해도 되고, * 을 통해 모든 컬럼 조회도 가능
    - * 
        - 테이블의 모든 데이터 출력
        - 예) SELECT * 
                FROM author;
    - ALL
        - 중복 데이터 모두 출력 (default)
        - 문법: SELECT [ALL] 컬럼/필드명 FROM 테이블명
        - 예) SELECT [ALL] name, email 
                FROM author;
    - DISTINCT 
        - 중복되는 값 제거
        - 문법: SELECT DISTINCT 컬럼명1 [, 컬럼명2, 컬럼명3 ...] FROM 테이블명;
        - DISTINCT 키워드는 SELECT 뒤와 첫 컬럼의 앞에 위치해야 함
        - NULL 값도 하나의 값으로 간주함
            - ORACLE에서 NULL값은 가장 큰 값으로 간주됨 -> ORDER BY 했을 때 가장 큰 값으로 간주됨
        - 예) SELECT DISTINCT name
                FROM author;
            - 중복된 name들 제거되고 하나씩만 반환
        - 컬럼이 하나 이상인 경우 모든 컬름들의 조합을 출력
        - 예) SELECT DISTINCT name, email 
                FROM author;
            - name과 email의 가능한 조합 반환
    - ROWNUM
        - TOP N개의 레코드를 반환
            - 상위 N개이므로 RONUM 뒤에는 =, >, >= 사용 불가; 다만, 1인 경우만 = 사용 가능
                - 얘) SELECT 컬럼명
                        FROM 테이블명
                        WHERE ROWNUM = 1; --> 상위 1개 출력
            - 오로지 RONUM <= 숫자, RONUM < 숫자 만 사용 가능
        - 사용자가 아닌 시스템이 관리하는 Pseudo Column (i.e., 쓸 수는 있지만 진짜 컬럼은 아닌 컬럼)
        - 채번, 출력 개수 지정 등에 활용 가능
        - 예)SELECT 컬럼명
                FROM 테이블명
                WHERE ROWNUM <= 숫자;
        - 주의: 다른 database system들은 다른 키워드를 사용한다
            - SQL Server / MS Access:
                - 문법: SELECT TOP 숫자 컬럼명
                        FROM 테이블명;
                - percent도 가능: SELECT TOP 숫자 PERCENT 컬럼명
                                    FROM 테이블명;
            - MySQL:
                - 믄밥: SELECT 컬럼명
                        FROM 테이블명
                        WHERE 조건
                        LIMIT 숫자;
            - Oracle 12:
                - 문법: SELECT 컬럼명
                        FROM 테이블명
                        ORDER BY 컬럼명
                        FETCH FIRST 숫자 ROWS ONLY;
                - percent도 가능: SELECT 컬럼명
                                    FROM 테이블명
                                    ORDER BY 컬럼명
                                    FETCH FIRST 숫자 PERCENT ROWS ONLY;
            - Older Oracle:
                - 문법: SELECT 컬럼명
                        FROM 테이블명
                        WHERE ROWNUM <= 숫자;
                - 문법 (with ORDER BY): SELECT 컬럼명
                                        FROM (SELECT 컬럼명 FROM 테이블명 ORDER BY 컬럼명)
                                        WHERE ROWNUM <= 숫자;
    - WHERE ~
        - 조건 걸기, 특정 조건을 만족하는 데이터 한정 출력
        - 문법: SELECT 컬럼명 FROM 테이블명 WHERE 조건
            - WHERE 컬럼명 (연산자) 컬럼값
        - 예) ... WHERE 컬럼명 = 컬럼값
    - ORDER BY
        - 선택한 결과의 정렬 (항상 조건 다 걸고 마지막에 붙임)
        - 문법: ORDER BY 컬럼명 [ASC/DESC]
        - SELECT 문으로 선택한 결과를 ORDER BY 절을 사용하여 정렬
        - 기본 설정은 ASC
        - 여러 컬럼의 데이터를 쉼표(,)를 사용하여 한 번에 정렬 가능
        - 예) SELECT * 
                FROM author 
                ORDER BY id DESC;
            - id를 기준으로 내림차순 정렬
        - 예) SELECT * 
                FROM author 
                ORDER BY name [ASC];
            - name을 기준으로 오름차순 정렬 [ASC]는 optional
        - 예) SELECT name, email
                FROM author
                ORDER BY 2;
            - 컬럼이 하나 이상일 경우 처음 컬럼부터 번호가 매겨진다 (name = 1, email = 2, ...)
            - ORDER BY 2에서 '2'는 SELECT 뒤 두번째로 명시된 컬럼인 'email'을 가리킨다
            - 그러므로 email을 기준으로 오름차순 정렬
        - ORDER BY절 생략시 PK를 기준으로 정렬하여 결과값 반환
        - ORDER BY절에는 SELECT절에 없는 컬럼도 사용 가능
        - 예) SELECT name 
                FROM author 
                ORDER BY email;
            - email을 기준으로 author 테이블의 name들이 오름차순 정렬로 출력
        - 예제) name만을 기준으로 오름차순 정렬
            - SELECT * FROM author ORDER BY name ASC;
        - 예제) name, email 기준으로 오름차순 정렬; name 중복일경우, email을 기준으로 정렬하게 되는 것
            - SELECT * FROM author ORDER BY name, email;
        - 예제) name은 오름차순, email은 내림차순으로 정렬
            - SELECT * FROM author ORDER BY name ASC, email DESC;
    - GROUP BY
        - 선택된 레코드의 집합을 툭정조건으로 그룹화한 결과 집합
        - 데이터의 값을 집계, 주로 집계 함수와 같이 사용
        - 집계 함수 예) SUM(), COUNT(), AVG(), MIN(), MAX() 등
        - 예) 작가가 쓴 글의 갯수를 알아보기; 이름, 이메일, 글 쓴 갯수 출력
            - SELECT a.name AS author_name, a.email, 
                        COUNT(p.author_id) AS 'post_count' 
                FROM author a 
                LEFT JOIN post p ON a.id = p.author_id 
                GROUP BY a.email;
        - 예) sales테이블에서 지점별(지점ID) 평균매출 구하기
            - SELECT branch_id, AVG(amount) AS average 
                FROM sales 
                GROUP BY branch_id;
    - GROUP BY + HAVING 조건절
        - 구문 전체에 대한 조건절은 WHERE ~ 이고 그룹화된 데이터에 대한 조건절은 HAVING ~
        - 예) author에서 중복되는 이름과 중복되는 횟수 조회
            - SELECT name, COUNT(*) AS COUNT 
                FROM author 
                GROUP BY name 
                HAVING COUNT(*) > 1;
        - 예) 2022년 한해동안 지점별 총매출을 구하라. 정렬은 금액 내림차순. 총매출이 2000 이상인 지점만 출력. 출력 - 지점명, 총매출액
            - SELECT b.name, SUM(s.amount)
                FROM branches AS b 
                INNER JOIN sales AS s 
                ON b.id = s.branch_id 
                WHERE s.created_date >= '2022-01-01' 
                AND s.created_date < '2023-01-01' 
                GROUP BY b.name 
                HAVING SUM(s.amount) >= 2000.00;
    - AS
        - 테이블과 필드에 임시로 별칭(alias)을 부여하고, 해당 별칭을 SELECT문에서 사용
        - AS 라는 키워드를 줘서 별칭(alias)를 이용한 처리 
        - 문법: SELECT 컬럼명 [AS] 별칭 FROM 테이블이름;
        - 생략 가능
        - 예) SELECT COUNT(DISTINCT name) AS count_name 
                FROM author;
        - 예) SELECT COUNT(DISTINCT name) count_name 
                FROM author;
        - 출력되는 별칭은 WHERE, FROM, GROUP BY, HAVING, ORDER BY 뒤에는 사용하지 않는다; 임시로 부여한 것이기 때문에 명령어 안에서는 사용되지 않는다
        - 별칭이 공백, 특수문자 등 포함 시 큰 따옴표("") 사용
            - 공백이나 특수문자 포함 안해도 "" 사용해도 됨
        - 예) SELECT name AS "작가 이름",
                        date AS "날짜!"
                        content AS "내용"
                FROM author
- INSERT INTO 
    - 테이블 한 건의 레코드 추가
        - 문법1: INSERT INTO 테이블명 (필드명1, 필드명2, 필드명3 ...) VALUES (데이터값1, 데이터값2, 데이터값3 ...);  -> 필드타입 넣을 필요 없음
            - 예) INSERT INTO author(id, name, email) values (1, 'leo', 'leo@gmail.com');
            - 컬럼 순서는 실제 테이블의 컬럼 순서와 무관
            - 정의하지 않은 컬럼은 NULL이 입력됨
        - 문법2: INSERT INTO 테이블명 VALUES (전체 컬럼의 value_list)
            - 예) INSERT INTO Pearl VALUES (13, 'merona', 'binggrae', 'melon', 600)
            - 전체 컬럼의 모든 값을 순서대로 입력해야 됨
            - 빈 값은 NULL 로 입력
                - 주의: MySQL에서는 안 되지만 ORACLE에서는 NULL 또는 작은 따옴표 ('')로 입력 가능 (**주의: 작은 따옴표 안에 공백이 있으면 안 됨 - 예) ' '는 하나의 공백 값으로 인식)
    - 여러 건 입력 시 INSERT ALL ~
        - 언제 사용?
            - 테이블 생성 후 초기 데이터 일괄 업로드
                - 문법: INSERT ALL
                        INTO 테이블명 VALUES (전체 컬럼의 value_list)
                        INTO 테이블명 VALUES (전체 컬럼의 value_list)
                        INTO 테이블명 VALUES (전체 컬럼의 value_list)
                        SELECT * FROM DUAL;
            - 기존 테이블의 레코드 조회 후 다른 테이블에 삽입 (예, 회원 명단 조회해서 그 중 미납 회원을 뽑아서 별도의 미납자 테이블 생성)
                - 문법: INSERT ALL
                                INTO 테이블2 VALUES (컬럼1, 컬럼2, ...)
                                INTO 테이블3 VALUES (컬럼1, 컬럼3, ...)
                        SELECT 컬럼1, 컬럼2, 컬럼3 FROM 테이블1
                    - 테이블1에서 레코드를 반환하여 테이블2, 테이블3에 분할 저장
        - 반드시 SELECT 구문도 함께 사용해야 됨 (SELECT 없을 시 에러 남)
    - 문자 또는 날짜 값은 작은 따옴표로 묶음 (숫자 데이터는 작은 따옴표 없이 사용)
- UPDATE 
    - 데이터 수정
    - 문법: UPDATE 테이블명 SET 필드명1 = 수정값1, 필드명2 = 수정값2 [WHERE 필드명 = 데이터값];
    - 조건이 없으면 다 바꿈 (조심해야 함)
    - 예) UPDATE author 
            SET email = 'pearl.ahn@gmail.com' 
            WHERE id = '2';
        - author 테이블의 id가 '2'인 레코드의 email을 'pearl.ahn@gmail.com'으로 바꿈
    - 예) UPDATE author
            SET email IS NULL
        - author 테이블의 모든 레코드의 email을 NULL값으로 바꿈
    - UPDATE와 ROWNUM을 이용한 채번
        - 테이블 내의 UNIQUE한 일련번호를 생성하는 경우
        - 1단계: ROWNUM을 담을 새 컬럼을 만든다
            - 예) ALTER TABLE 테이블명 ADD (새컬럽명 NUMBER);
                    ALTER TABLE Pearl ADD (ROW_ID NUMBER)
                - 'Pearl'이라는 테이블에 'ROW_ID'의 이름을 가지고 NUMBER라는 타입을 가진 빈 컬럼 추가
        - 2단계: 해당 컬럼에 ROWNUM값을 준다
            - 예) UPDATE 테이블명 SET 새컬럼명 = ROWNUM;
                    UPDATE Pearl SET ROW_ID = ROWNUM;
                - 'ROW_ID'라는 컬럼에 해당 레코드의 ROWNUM 값으로 수정
- DELETE
    - 데이터 삭제 (테이블에 존재하는 전체 레코드 삭제)
    - 문법: DELETE [FROM] 테이블명 [WHERE 필드명 = 데이터값];
        - DELETE 테이블명; 또는 DELETE FROM 테이블명
        - 주의: MySQL에서는 꼭 FROM을 써줘야 된다 -> DELETE FROM ~
    - 한 row 전체를 지우기 때문에 컬럼명이나 *를 지정해줄 필요 없다
    - 삭제 후 복구 가능 (TRUNCATE는 복구 불가)
    - 조건을 빼버리면 테이블 전체를 지움 (조심해야 됨)
    - 예) DELETE FROM author 
            WHERE id >= 10;
DCL (Data Control Lang) 데이터 제어어: 사용자 권한 관리 (계정 생성 후 권한부여)
- GRANT (권한 부여)
- REVOKE (권한 취소)
+ TCL (Transaction Control Lan) 트랜젝션 제어어: 변경 내용을 확정/취소
- COMMIT
- ROLLBACK

함수 유형
- 생성 주체
    - 사용자 정의 함수 (User Defined Function) - 사용자가 정의
    - 내장 함수 (Built-in Function) - 벤더가 정의 
        - 예) Oracle에서 SUM(), LENGTH() 등등
- 적용 범위
    - 단일 행 함수 (Single-Row Function)
        - 문자형 함수, 숫자형 함수, 날짜형 함수
        - 제어 함수, 변환 함수, NULL 관련 함수
        - 특징:
            - 각 행(row)에 대해 개별적으로 작용하고 그 결과를 반환
                - 단일 행 내에 있는 하나/복수의 값을 인수로 사용
                - 여러 행에 걸친 값을 사용할 수 없음
            - 함수 중첩 (함수의 인자로 함수를 사용) 가능
            - SELECT, WHERE, ORDER BY 절에 사용 가능
        - 예) SELECT player_name, LENGTH(player_name) AS 길이
                FROM player;
                >> 'player_name'과 '길이'라는 필드이름을 가진 테이블이 출력되고, 각 행에는 player_name과 player_name의 길이가 숫자로 담긴 값들이 담겨져 있을 것
                >> 예) player_name이 '윤희준'이면 길이는 3
        - Oracle의 단일행 내장 함수
            - 문자형 함수: 문자형 변수 처리 
                - 예) LOWER(), UPPER(), ASCII(), CHR(), CONCAT(), SUBSTR(), LENGTH(), LTRIM(), RTRIM(), TRIM()
            - 숫자형 함수: 숫자형 변수 처리
                - 예) ABS(), SIGN(), MOD(), CEIL(), FLOOR(), ROUND(), TRUNC(), SIN(), COS(), TAN(), EXP(), POWER(), SQRT(), LOG(), LN()
            - 변환 함수: 문자, 숫자, DATE형 값의 타입 변환
                - 예) TO_CHAR(), TO_NUMBER(), TO_DATE()
            - 날짜형 함수: DATE 타입의 변수 처리
                - 예) SYSDATE(), EXTRACT(), TO_NUMBER(TO_CHAR(d,'DD'|'MM'|'YY'))
            - 제어 함수: 논리값에 따른 값의 처리
                - 예) CASE, DECODE
            - NULL 관련 함수: NULL 처리
                - 예) NVL(), NULLIF(). COALESCE()
    - 다중 행 함수 (Multi-Row Function)
        - 그룹 함수 (Group Function)
            - 집계 함수 (Aggregate Function) 포함
        - 윈도우 함수 (Window Function)